# Pricing Automation Detection

In [1]:
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import grangercausalitytests
import numbers
import statistics as st


df = pd.read_csv('final_csv_task1.csv', usecols=['comp_date',
                                                 'product_id',
                                                 'seller_id',
                                                 'min',
                                                 'price',
                                                 'max' ,
                                                 'difference'])

enc = LabelEncoder()
df['enc_comp_date'] = enc.fit_transform(df['comp_date'])

df["comp_date"] =  pd.to_datetime(df["comp_date"], format = "%Y/%m/%d")
df["quarter"] = df.comp_date.dt.quarter

def pivot(product_id, quarter):
    df_1 = df[(df.product_id == product_id) & (df.quarter == quarter)]
    table = pd.pivot_table(df_1, values='price', index=['comp_date'], columns=['seller_id'])
    table.fillna(method='ffill', inplace=True)
    return table

dfp = pivot(110064, 1)

In [2]:
dfp.reset_index('comp_date', drop = True, inplace = True)

In [3]:
dfp.fillna("null", inplace = True)

In [4]:
dizionario = dict()

for sell in dfp:
    lista_seller = []
    for i in range(1, len(dfp)):

        if dfp.loc[i, sell] != dfp.loc[i-1, sell] and (dfp.loc[i, sell] and dfp.loc[i-1, sell]) != 'null':
            lista_seller.append(i)
    dizionario[sell] = lista_seller

In [5]:
for i in dizionario:
    print(i, dizionario[i])

23 []
24 [7, 13, 14, 17, 26, 27, 28, 31, 32, 33, 35, 39, 41, 48, 52, 55, 57, 59, 60, 61, 62, 66, 70, 71, 73, 77, 84, 87]
26 [10, 13, 21, 24, 25, 26, 31, 32, 39, 41, 42, 49, 52, 57, 59, 60, 63, 66, 68, 69, 70, 71, 73, 75, 76, 80, 81, 87]
41 [26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 42, 43, 47, 48, 50, 52, 53, 56, 57, 61, 62]
48 [31, 32, 59, 60, 75, 76, 77, 87]
180 [6, 7, 33, 34, 35, 36, 37, 38, 40, 43, 44, 47, 48, 50, 51, 52, 53, 54, 56, 57, 58, 59, 62, 63, 88, 89]
407 [12, 15, 17, 18, 19, 20, 26, 36, 37, 49, 62, 63, 88]
490 []


In [6]:
dict_all = dict()

for i in dizionario:
    if len(dizionario[i]) != 0:
        dict_diff = dict()
        
        for j in dizionario: 
            if j != i:
                if len(dizionario[j]) != 0:
                    lista_diff = []
                    for elem_1 in range(len(dizionario[i])):
                        list_temp = [item for item in dizionario[j] if item <= dizionario[i][elem_1]]
                        if len(list_temp) != 0:
                            diff = dizionario[i][elem_1] - max(list_temp)
                            lista_diff.append(diff)
                        else: lista_diff.append('no recent date')
                        
                        dict_diff[j] = lista_diff
        dict_all[i] = dict_diff

In [7]:
for i in dict_all:
    print(i, dict_all[i])

24 {26: ['no recent date', 0, 1, 4, 0, 1, 2, 0, 0, 1, 3, 0, 0, 6, 0, 3, 0, 0, 0, 1, 2, 0, 0, 0, 0, 1, 3, 0], 41: ['no recent date', 'no recent date', 'no recent date', 'no recent date', 0, 0, 0, 0, 1, 0, 0, 2, 4, 0, 0, 2, 0, 2, 3, 0, 0, 4, 8, 9, 11, 15, 22, 25], 48: ['no recent date', 'no recent date', 'no recent date', 'no recent date', 'no recent date', 'no recent date', 'no recent date', 0, 0, 1, 3, 7, 9, 16, 20, 23, 25, 0, 0, 1, 2, 6, 10, 11, 13, 0, 7, 0], 180: [0, 6, 7, 10, 19, 20, 21, 24, 25, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 3, 7, 8, 10, 14, 21, 24], 407: ['no recent date', 1, 2, 0, 0, 1, 2, 5, 6, 7, 9, 2, 4, 11, 3, 6, 8, 10, 11, 12, 0, 3, 7, 8, 10, 14, 21, 24]}
26 {24: [3, 0, 4, 7, 8, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 2, 3, 0, 0, 0, 2, 3, 3, 4, 0], 41: ['no recent date', 'no recent date', 'no recent date', 'no recent date', 'no recent date', 0, 0, 1, 2, 4, 0, 1, 0, 0, 2, 3, 1, 4, 6, 7, 8, 9, 11, 13, 14, 18, 19, 25], 48: ['no recent date', 'no recent date', 'no recent dat

In [8]:
df = pd.DataFrame(0, index=list(dizionario.keys()), columns=list(dizionario.keys()))

for col in df.columns:
    for key1,v1 in dict_all.items():
        if col != key1:
            for key2,v2 in v1.items():
                st_dv = st.stdev([x for x in v2 if isinstance(x, numbers.Number)])
                df.loc[key1, key2] = st_dv

In [9]:
df

,23,24,26,41,48,180,407,490
23,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
24,0,0.000000,1.556166,7.126192,8.064325,9.073699,6.063294,0
26,0,2.185813,0.000000,7.153042,7.094446,8.359245,5.945143,0
41,0,1.376991,1.874874,0.000000,8.889022,9.746683,3.800205,0
48,0,1.187735,0.353553,8.967202,0.000000,9.934608,5.841661,0
180,0,1.405940,1.880487,7.381921,8.987511,0.000000,3.829708,0
407,0,1.386750,2.396579,9.712535,5.887841,6.486673,0.000000,0
490,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
